In [ ]:
from flask import *
from langchain import OpenAI,PromptTemplate,LLMChain
import openai


In [ ]:
#api 키 불러오기
def load_api_key(file_path):
    with open(file_path, 'r') as file:
        api_key = file.read().strip()
    return api_key

# OpenAI API 설정
api_key = load_api_key("") #api 파일 경로 설정
model = OpenAI(openai_api_key=api_key, model="GPT-4o")

# 타로+사용자고민 해석 함수
def interpret_cards(user_input,cards):
    try:    
        tarot_template_reason = PromptTemplate(
            template=(
                "사용자의 질문: '{user_input}'\n"
                "타로카드: {cards}\n"
                "사용자의 질문과 타로카드를 통해 심리상담의 역할을 해주고, 타로카드를 중심으로 해석해주세요."
            ),
            input_variables=["user_input","cards"]
        )

        # LLMChain 생성
        reason_chain = LLMChain(llm=model,prompt_template=tarot_template_reason)
        reason_response = reason_chain.run(input=user_input,cards=cards)

        answer = reason_response.strip()
        
        # 답변 길이 제한
        if len(answer) > 500:
            answer = answer[:500]

        return jsonify({"answer":answer})

    except Exception as e:
        return "해석 중 오류가 발생했습니다."


# 타로 일반 해석 함수
def general_reading(user_select,cards):
    try:
        tarot_template_genal = PromptTemplate(
        template=(
            "사용자의 질문 :'{user_select}'\n"# 오늘의 운세, 연애, 금전, 직장
            "타로카드:{cards}\n",
            "사용자의 {user_select}를 주제로 타로 해석을 해주세요"
            ),
        input_variables=["user_select","cards"]
        )
        general_chain = LLMChain(llm=model,prompt_template=tarot_template_genal)
        general_response = general_chain.run(input=user_select,cards=cards)
        answer = general_response.strip()
        
        # 답변 길이 제한
        if len(answer) > 500:
            answer = answer[:500]

        return jsonify({"answer":answer})
    
    except Exception as e:
        return "해석 중 오류가 발생했습니다."